In [2]:
import zipfile
with zipfile.ZipFile('ECHR_Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('cases')

OSError: [Errno 122] Disk quota exceeded

In [1]:
import pandas as pd
import json
import os 

directory = 'cases/EN_dev'
df = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('json'):
       file_path = (os.path.join(directory,filename))
       with open(file_path, 'r') as f:
            data = json.load(f)             
              
       temp_df = pd.json_normalize(data)
     
       df = pd.concat([df, temp_df], ignore_index=True)
        


print(df.shape)
#file_path = 'cases/training_set_tweets.txt'
#df = pd.read_csv(file_path, delimiter='\t', header=None, on_bad_lines='skip')

#df_short = df.head(5000).dropna()  #iloc[:, 2]
#df_short.columns=['a', 'b', 'Text', 'd']
#print(df.shape)
#print(df_short.shape)
#print(df_short.columns)
#print(df_short.Text)
#print(df_short.isna().sum())


(1380, 16)


In [20]:
import numpy as np
import tiktoken

arr = np.zeros(1380)
rows = df.iterrows()
filtered_rows = pd.DataFrame(columns=df.columns)
for index,row in rows:
    encoding = tiktoken.get_encoding('cl100k_base')
    text = " ".join(" ".join(row['TEXT']).split())
    text = text.replace("\n", " ")
    num_tokens = len(encoding.encode(text))
    if(num_tokens < 8000):
        filtered_rows.loc[len(filtered_rows)] = row
    arr[index] = num_tokens


print(min(arr))
print(sum(arr))

TypeError: '<' not supported between instances of 'list' and 'int'

In [34]:
print("SHAPE OF DF: " + str(filtered_rows.shape))

num_tokens_cum = 0

for index,row in filtered_rows.iterrows():
    encoding = tiktoken.get_encoding('cl100k_base')
    text = " ".join(" ".join(row['TEXT']).split())
    text = text.replace("\n", " ")
    num_tokens = len(encoding.encode(text))
    num_tokens_cum += num_tokens
print("NUMBER OF CUMULATIVE TOKENS IN DF TEXT: " + str(num_tokens_cum))

df_short = filtered_rows.head(300)

num_tokens_cum = 0
for index,row in df_short.iterrows():
    encoding = tiktoken.get_encoding('cl100k_base')
    text = " ".join(" ".join(row['TEXT']).split())
    text = text.replace("\n", " ")
    num_tokens = len(encoding.encode(text))
    num_tokens_cum += num_tokens
print("NUMBER OF CUMULATIVE TOKENS IN DF TEXT: " + str(num_tokens_cum))



SHAPE OF DF: (1276, 16)
NUMBER OF CUMULATIVE TOKENS IN DF TEXT: 3034685
NUMBER OF CUMULATIVE TOKENS IN DF TEXT: 730180


In [30]:
print(df_short.columns)

count_nan_rows = df_short.isna().sum(axis=1).sum()

print(count_nan_rows)  

Index(['ITEMID', 'LANGUAGEISOCODE', 'RESPONDENT', 'BRANCH', 'DATE', 'DOCNAME',
       'IMPORTANCE', 'CONCLUSION', 'JUDGES', 'TEXT', 'VIOLATED_ARTICLES',
       'VIOLATED_PARAGRAPHS', 'VIOLATED_BULLETPOINTS', 'NON_VIOLATED_ARTICLES',
       'NON_VIOLATED_PARAGRAPHS', 'NON_VIOLATED_BULLETPOINTS'],
      dtype='object')
0


In [35]:
import openai
from openai.embeddings_utils import get_embedding
model='text-embedding-ada-002'

openai.api_key = '**'
i = iter(list(range(1, 10000)))

def get_embedding(text, model="text-embedding-ada-002"):
   print(next(i))
   text = " ".join(" ".join(text).split()[:100])
   text = text.replace("\n", " ")
   print(len(text))
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']


df_short['EMBEDDING'] = df_short['TEXT'].apply(lambda x: get_embedding(x, model))


new_df = df_short.iloc[:, [0, 16]]
print(df_short.shape)
print(df_short['EMBEDDING'])
new_df.to_csv('embedded_cases.csv', index=False)

1
586
2
637
3
580
4
607
5
612
6
644
7
599
8
625
9
613
10
612
11
532
12
579
13
579
14
551
15
640
16
577
17
529
18
582
19
584
20
580
21
583
22
603
23
633
24
543
25
607
26
610
27
625
28
627
29
629
30
566
31
564
32
564
33
590
34
569
35
541
36
575
37
624
38
622
39
574
40
568
41
601
42
545
43
595
44
559
45
547
46
572
47
572
48
574
49
572
50
601
51
614
52
610
53
606
54
630
55
649
56
547
57
585
58
635
59
636
60
572
61
587
62
548
63
594
64
574
65
622
66
590
67
599
68
582
69
554
70
587
71
575
72
576
73
600
74
649
75
543
76
562
77
567
78
582
79
621
80
621
81
558
82
599
83
586
84
562
85
589
86
616
87
559
88
569
89
592
90
572
91
613
92
446
93
624
94
605
95
579
96
581
97
574
98
584
99
586
100
600
101
583
102
603
103
593
104
582
105
602
106
593
107
592
108
675
109
583
110
632
111
558
112
598
113
602
114
650
115
567
116
606
117
601
118
561
119
581
120
647
121
578
122
599
123
608
124
578
125
547
126
575
127
584
128
619
129
556
130
575
131
563
132
554
133
610
134
582
135
559
136
603
137
602
138
525
139


/tmp/ipykernel_615/3365657501.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['EMBEDDING'] = df_short['TEXT'].apply(lambda x: get_embedding(x, model))


(300, 17)
0      [-0.00242983247153461, -0.0018658250337466598,...
1      [0.001230870489962399, 0.0004578838124871254, ...
2      [-0.0035945656709372997, 0.0070950682274997234...
3      [0.00996319018304348, 0.014701299369335175, -0...
4      [0.002267291536554694, -0.01349721010774374, -...
                             ...                        
295    [0.003906595055013895, 0.0019697118550539017, ...
296    [-0.0012068739160895348, 0.014544841833412647,...
297    [0.0035572857595980167, -0.029464254155755043,...
298    [0.010518673807382584, 0.0008521137060597539, ...
299    [0.008434606716036797, -0.005519558675587177, ...
Name: EMBEDDING, Length: 300, dtype: object


In [60]:
print(df_short.shape)
print(df_short['ITEMID'][0])

(100, 17)
001-59587


In [61]:
print(df_short)

        ITEMID LANGUAGEISOCODE RESPONDENT         BRANCH  DATE  \
0    001-59587             ENG        FIN   GRANDCHAMBER  2001   
1    001-69131             ENG        POL        CHAMBER  2005   
2    001-88827             ENG        HUN  ADMISSIBILITY  2008   
3   001-106159             ENG        MDA  ADMISSIBILITY  2011   
4   001-115325             ENG        POL  ADMISSIBILITY  2012   
..         ...             ...        ...            ...   ...   
95  001-112141             ENG        FIN  ADMISSIBILITY  2012   
96   001-57612             ENG        ITA        CHAMBER  1989   
97  001-107139             ENG        SVN        CHAMBER  2011   
98   001-22657             ENG        TUR  ADMISSIBILITY  2002   
99   001-23945             ENG        SVK  ADMISSIBILITY  2004   

                                 DOCNAME IMPORTANCE  \
0           CASE OF K. AND T. v. FINLAND          1   
1        CASE OF J.S. AND A.S. v. POLAND          3   
2                      JARFAS v. HUNGARY  

In [84]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



def string_from_arr(text):
     text = " ".join(" ".join(text).split()[:100])
     return text.replace("\n", " ")

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

input_sentence = """This case revolves around a small community residing near a large industrial facility. Residents have experienced adverse health effects and witnessed environmental degradation, potentially associated with the facility's operations. The objective is to uncover the truth, identify responsible parties, and pursue justice on behalf of the affected community.
The community, located in a tranquil countryside setting, has historically enjoyed a peaceful existence. However, since the establishment of the industrial facility a few years ago, residents have reported a rise in health issues such as respiratory ailments, skin conditions, and chronic illnesses. They have also observed the deterioration of local flora and fauna, as well as signs of contamination in nearby water sources.
Initial investigations revealed elevated levels of pollutants in soil, air, and water samples obtained from the vicinity of the facility. These findings suggest a possible correlation between the facility's activities and the adverse effects experienced by the community.
To gain further insights, renowned environmental scientists, toxicologists, and health experts were engaged. They conducted thorough analyses and studies to assess the extent of contamination, identify potential health risks, and determine the likely source of pollutants. Their expertise is crucial in substantiating claims and guiding legal action.
Identifying the responsible parties is a key step in this case. While the primary accountability lies with the industrial facility due to its proximity and suspected connection to the adverse effects, it is essential to consider the involvement of third-party contributors or regulatory oversights.
In collaboration with community advocates, affected individuals are encouraged to share their experiences and concerns. This collective effort aims to raise public awareness, foster unity, and empower the community.
With compelling evidence, expert opinions, and testimonies from affected residents, a strong legal case is being built. The objective is to demonstrate the facility's negligence, its impact on the environment and public health, and the hardships faced by the community.
Legal action seeks comprehensive resolutions, including monetary compensation for affected individuals, implementation of stringent environmental regulations, and sustainable remediation efforts to restore the local ecosystem.
This case of environmental contamination underscores the importance of environmental stewardship, accountability, and protecting community well-being. The pursuit of justice aims to secure a safer and healthier future for the affected community while striving to prevent similar incidents in the future."""

input_embedding = np.array(get_embedding(input_sentence, model))

df2 = pd.read_csv('embedded_cases.csv')

df2['EMBEDDING'] = df2['EMBEDDING'].apply(eval).apply(np.array, dtype=float)

embeddings = np.array(df2['EMBEDDING'].tolist())



similarities = cosine_similarity([input_embedding], embeddings)



sorted_indices = np.argsort(similarities[0])[::-1]




top_10_case_ids= [df2['ITEMID'][idx] for idx in sorted_indices[:10]]

df3 = pd.DataFrame(columns=['ITEMID', 'TEXT', 'SUMMARY'])   

for id in top_10_case_ids[:3]:
    
    
    temp_df = df_short[df_short['ITEMID'] == id].copy()
    
    text = string_from_arr(temp_df['TEXT'].values[0])
    print(type(text))
    print(len(text))
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
           {"role": "system", "content": "You are a lawyer trying to write a concise summary of this case for other lawyers to read and compare to other cases."},
            {"role": "user", "content": "Summarize this text with explicit mentions of judicially relevant facts included, such as intoxication, fraud, mental illness, etc.:\n\n" + text},
        ]
    )
    
    print(completion.choices[0].message.content)
    
    temp_df.loc[:, 'SUMMARY'] = completion.choices[0].message.content
    
    df3 = pd.concat([df3, temp_df.iloc[:, [0, 9, 17]]], ignore_index=True)
    

    
df3.to_csv('found_cases.csv', index=False)
    



<class 'str'>
635
This case involves a 56-year-old applicant who was the editor-in-chief of a local newspaper. On October 17, 2005, the newspaper published an article, coauthored by the applicant, titled "Municipality in danger; authorities fail to see problem". The article discussed the inadequate sewage system in the Iława municipality, emphasizing the public health risks and the need for extensive investments to improve it. The case is relevant to judicial matters such as freedom of the press and potential liability for the dissemination of information. No information regarding intoxication, fraud, or mental illness is mentioned.
<class 'str'>
598
This case involves the first applicant, who owned three freshwater fish farms, being charged with offences against the Act on freshwater fish farms. The charges include intentionally exceeding fixed feed quotas, with the potential danger or risk to the environment, as well as enrichment for himself. The trial was initially scheduled for 9 

In [52]:
df3 = pd.DataFrame(columns=['TEXT', 'SUMMARY'])     
for id in top_10_case_ids:
    
    
    temp_df = df_short[df_short['ITEMID'] == id]

    print(type(temp_df))
    print(temp_df.shape)
    print(temp_df.columns)
    
    text = string_from_arr(temp_df['TEXT'].values[0])
    #print(type(text))
    #completion = openai.ChatCompletion.create(
     #   model="gpt-3.5-turbo",
      #  messages=[
       #     {"role": "system", "content": "You are a lawyer trying to write a concise summary of this case for other lawyers to read"},
        #    {"role": "user", "content": "Summarize this text:\n\n" + text},
        #]
    #)
    
    #print(completion.choices[0].message)
    
    #temp_df['SUMMARY'] = completion.choices[0].message
    
    
    #df3 = pd.concat([df3, temp_df.iloc[:, 9, 16]], ignore_index=True)
    
    
#df3.to_csv('found_cases.csv', index=False)
    


NameError: name 'top_10_case_ids' is not defined